# 05 CNN 网络架构

> 组合卷积层、池化层、全连接层，构建完整 CNN

---

## 📚 学习目标

- [ ] 理解 CNN 的标准架构模式
- [ ] 追踪特征图形状变化
- [ ] 从零实现完整的 CNN 类
- [ ] 使用 PyTorch 构建 CNN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 尝试导入 PyTorch
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    HAS_TORCH = True
    print("PyTorch 已导入")
except ImportError:
    HAS_TORCH = False
    print("PyTorch 未安装，部分代码无法运行")

np.random.seed(42)

---

## 💡 CNN 的标准架构

```
输入图像
    ↓
┌─────────────────────────────────┐
│  特征提取部分（重复多次）           │
│  Conv → ReLU → Pool              │
│  Conv → ReLU → Pool              │
│  ...                             │
└─────────────────────────────────┘
    ↓
  Flatten（展平）
    ↓
┌─────────────────────────────────┐
│  分类部分                         │
│  FC → ReLU → Dropout            │
│  FC → Softmax                   │
└─────────────────────────────────┘
    ↓
  输出（类别概率）
```

---

## 📊 特征图形状变化追踪

以 MNIST (28×28×1) 为例：

In [ ]:
def calc_output_size(H_in, k, p=0, s=1):
    """计算卷积/池化后的输出尺寸"""
    return (H_in + 2*p - k) // s + 1

# MNIST 图像处理流程
print("MNIST CNN 形状变化追踪")
print("=" * 60)

# 初始输入
H, W, C = 28, 28, 1
print(f"输入: {C}×{H}×{W}")

# Conv1: 1→32, 3×3, same padding
H = calc_output_size(H, 3, p=1)
C = 32
print(f"Conv1 (32, 3×3, p=1): {C}×{H}×{H}")

# Pool1: 2×2
H = calc_output_size(H, 2, s=2)
print(f"MaxPool (2×2): {C}×{H}×{H}")

# Conv2: 32→64, 3×3, same padding
H = calc_output_size(H, 3, p=1)
C = 64
print(f"Conv2 (64, 3×3, p=1): {C}×{H}×{H}")

# Pool2: 2×2
H = calc_output_size(H, 2, s=2)
print(f"MaxPool (2×2): {C}×{H}×{H}")

# Flatten
flat = C * H * H
print(f"Flatten: {flat}")

# FC
print(f"FC1: {flat} → 128")
print(f"FC2: 128 → 10 (输出类别)")

---

## 💻 NumPy 实现完整 CNN

In [ ]:
class SimpleCNN:
    """
    简单的 CNN 实现（仅前向传播）
    
    架构:
        Conv(1→8, 3×3) → ReLU → MaxPool(2×2)
        Conv(8→16, 3×3) → ReLU → MaxPool(2×2)
        Flatten → FC(400→64) → ReLU → FC(64→10)
    """
    
    def __init__(self):
        # 初始化权重（随机）
        # Conv1: 8 个 3×3 核，1 个输入通道
        self.conv1_w = np.random.randn(8, 1, 3, 3) * 0.1
        self.conv1_b = np.zeros(8)
        
        # Conv2: 16 个 3×3 核，8 个输入通道
        self.conv2_w = np.random.randn(16, 8, 3, 3) * 0.1
        self.conv2_b = np.zeros(16)
        
        # FC1: 5×5×16 = 400 → 64
        self.fc1_w = np.random.randn(400, 64) * 0.1
        self.fc1_b = np.zeros(64)
        
        # FC2: 64 → 10
        self.fc2_w = np.random.randn(64, 10) * 0.1
        self.fc2_b = np.zeros(10)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def conv2d(self, x, w, b, padding=1):
        """简化的卷积（单样本）"""
        C_out, C_in, k_h, k_w = w.shape
        _, H, W = x.shape
        
        if padding > 0:
            x = np.pad(x, ((0,0), (padding,padding), (padding,padding)))
        
        out_h = H
        out_w = W
        output = np.zeros((C_out, out_h, out_w))
        
        for c_out in range(C_out):
            for i in range(out_h):
                for j in range(out_w):
                    window = x[:, i:i+k_h, j:j+k_w]
                    output[c_out, i, j] = np.sum(window * w[c_out]) + b[c_out]
        
        return output
    
    def max_pool(self, x, size=2):
        """最大池化"""
        C, H, W = x.shape
        out_h, out_w = H // size, W // size
        output = np.zeros((C, out_h, out_w))
        
        for c in range(C):
            for i in range(out_h):
                for j in range(out_w):
                    window = x[c, i*size:(i+1)*size, j*size:(j+1)*size]
                    output[c, i, j] = np.max(window)
        
        return output
    
    def forward(self, x):
        """
        前向传播
        
        参数:
            x: 输入图像, shape (1, 28, 28)
        
        返回:
            logits: 未归一化的类别分数, shape (10,)
        """
        # Conv1 → ReLU → Pool
        x = self.conv2d(x, self.conv1_w, self.conv1_b)  # (8, 28, 28)
        x = self.relu(x)
        x = self.max_pool(x)  # (8, 14, 14)
        
        # Conv2 → ReLU → Pool
        x = self.conv2d(x, self.conv2_w, self.conv2_b)  # (16, 14, 14)
        x = self.relu(x)
        x = self.max_pool(x)  # (16, 7, 7)
        
        # 展平
        x = x.flatten()  # (784,) 但我们设计为 (400,)
        # 注意：这里简化处理，实际尺寸可能不同
        
        # FC1 → ReLU
        x = x[:400]  # 截断到 400
        x = x @ self.fc1_w + self.fc1_b
        x = self.relu(x)
        
        # FC2
        x = x @ self.fc2_w + self.fc2_b
        
        return x

# 测试
cnn = SimpleCNN()
dummy_input = np.random.randn(1, 28, 28)
output = cnn.forward(dummy_input)
print(f"输入形状: {dummy_input.shape}")
print(f"输出形状: {output.shape}")
print(f"输出值: {output}")

---

## 💻 PyTorch 实现 CNN

In [ ]:
if HAS_TORCH:
    class MNISTNet(nn.Module):
        """
        MNIST 分类 CNN
        
        架构:
            Conv(1→32, 3×3) → ReLU → MaxPool(2×2)
            Conv(32→64, 3×3) → ReLU → MaxPool(2×2)
            FC(1600→128) → ReLU → Dropout
            FC(128→10)
        """
        
        def __init__(self):
            super().__init__()
            
            # 卷积层
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
            
            # 池化层
            self.pool = nn.MaxPool2d(2, 2)
            
            # 全连接层
            # 28→14→7, 所以展平后是 64×7×7 = 3136
            self.fc1 = nn.Linear(64 * 7 * 7, 128)
            self.fc2 = nn.Linear(128, 10)
            
            # Dropout
            self.dropout = nn.Dropout(0.5)
        
        def forward(self, x):
            # Conv1 → ReLU → Pool
            x = self.pool(F.relu(self.conv1(x)))  # (N, 32, 14, 14)
            
            # Conv2 → ReLU → Pool
            x = self.pool(F.relu(self.conv2(x)))  # (N, 64, 7, 7)
            
            # 展平
            x = x.view(-1, 64 * 7 * 7)  # (N, 3136)
            
            # FC1 → ReLU → Dropout
            x = self.dropout(F.relu(self.fc1(x)))  # (N, 128)
            
            # FC2
            x = self.fc2(x)  # (N, 10)
            
            return x
    
    # 测试
    model = MNISTNet()
    dummy = torch.randn(4, 1, 28, 28)  # batch_size=4
    out = model(dummy)
    print(f"输入形状: {dummy.shape}")
    print(f"输出形状: {out.shape}")
    
    # 打印模型结构
    print("\n模型结构:")
    print(model)
    
    # 参数量
    total_params = sum(p.numel() for p in model.parameters())
    print(f"\n总参数量: {total_params:,}")

---

## 📝 本章小结

### CNN 架构模式

```
[Conv → ReLU → Pool] × N → Flatten → [FC → ReLU → Dropout] × M → Output
```

### 设计原则

1. 空间尺寸逐渐减小（通过池化或 stride=2）
2. 通道数逐渐增加（32 → 64 → 128 → ...）
3. 最后用全连接或全局平均池化

---

## 🚀 下一步

继续学习 **06_cnn_backprop.ipynb** - CNN 反向传播